<a href="https://colab.research.google.com/github/DS3001/group9/blob/main/midterm_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependencies and load data

In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

import zipfile # for reading csv as zip

In [43]:
# zip file in git repo
zip_path = "/gss.csv.zip"

# Desired csv file in zip
file_to_read = "gss.csv"

with zipfile.ZipFile(zip_path, 'r') as zip_file:

  var_list = ['pres16', 'year', 'abmoral', 'age', 'degree', 'padeg', 'madeg', 'sex', 'race', 'wrkstat', 'wrkgovt'] # List of variables to save
  output_file = 'raw_gss_data.csv' # Name of the file to save the data to

  parser = pd.read_csv(zip_file.open(file_to_read), iterator=True, chunksize=10000, low_memory=False) # Creates a parser for the whole data
  modes = ['w','a'] # Has write mode and append mode
  phase = 0 # Starts in write mode; after one iteration of loop, switches to append mode

  for chunk in parser: # For each chunk of the data
      print(chunk.loc[:,var_list].head()) # Visually inspect the first few rows
      chunk.loc[:,var_list].to_csv(output_file, # specifies target file to save the chunk to
                                  mode=modes[phase], # control write versus append
                                  header=var_list, # variable names
                                  index=False) # no row index saved
      phase = 1 # Switch from write mode to append mode




   pres16  year  abmoral   age                 degree                  padeg  \
0     NaN  1972      NaN  23.0             bachelor's  less than high school   
1     NaN  1972      NaN  70.0  less than high school  less than high school   
2     NaN  1972      NaN  48.0            high school  less than high school   
3     NaN  1972      NaN  27.0             bachelor's             bachelor's   
4     NaN  1972      NaN  61.0            high school  less than high school   

                   madeg     sex   race            wrkstat  wrkgovt  
0                    NaN  female  white  working full time      NaN  
1  less than high school    male  white            retired      NaN  
2  less than high school  female  white  working part time      NaN  
3            high school  female  white  working full time      NaN  
4  less than high school  female  white      keeping house      NaN  
       pres16  year  abmoral   age                 degree  \
10000     NaN  1978      NaN  28.0    

In [44]:
# Create dataframe from 'raw_gss_data.csv' that was just written
gss_dat = pd.read_csv("raw_gss_data.csv")


# Data cleaning and preparation

In [35]:
print(gss_dat['pres16'].value_counts())

# Remove "didn't vote for president" and other no answer responses (see codebook)
gss_dat = gss_dat.loc[
    (gss_dat['pres16'] == 'clinton') |
    (gss_dat['pres16'] == 'trump') |
    (gss_dat['pres16'] == 'other candidate'), :]

# Every entry either voted for Trump, Clinton, or other unspecified candidate
print(gss_dat['pres16'].value_counts())
print(gss_dat['pres16'].isnull().sum())

clinton            3390
trump              2444
other candidate     400
Name: pres16, dtype: int64
clinton            3390
trump              2444
other candidate     400
Name: pres16, dtype: int64
0


In [34]:
gss_dat['year'].value_counts() # year data is good because of our filtering of pres16 (if they respond to having voted in 2016, they are being surveyed after 2016)
gss_dat['year'].isnull().sum()

0

In [36]:
gss_dat['abmoral'].value_counts() # clean, can remove "depends" for more specific results
gss_dat['abmoral'].isnull().sum() # so many absent values, no reason to continue with variable
gss_dat.drop('abmoral', axis=1, inplace=True) # drop column 'abmoral' on axis 1 (column)

In [41]:
gss_dat['age'].value_counts()
gss_dat['age'].isnull().sum()
gss_dat = gss_dat.loc[~gss_dat['age'].isnull(), ] # keep rows where age is NOT null
gss_dat['age'].isnull().sum()

0

In [58]:
gss_dat['degree'].isnull().sum()
gss_dat['degree'].value_counts()
gss_dat['degree'].unique() # NaN and 'degree' present

gss_dat = gss_dat.loc[(~gss_dat['degree'].isnull()) & (gss_dat['degree'] != 'degree'), ] # keep rows where value for degree is not NaN or 'degree' (see codebook, NaN and 'degree' respones are invalid)
gss_dat['degree'].unique() # all observations are now "bachelor's", 'less than high school', 'high school', 'graduate', or 'associate/junior college'



array(["bachelor's", 'less than high school', 'high school', 'graduate',
       'associate/junior college'], dtype=object)

In [62]:
gss_dat['madeg'].isnull().sum()
gss_dat['madeg'].unique() # NaNs present

gss_dat = gss_dat.loc[~gss_dat['madeg'].isnull(), ] # keep rows where value for madeg is not NaN
gss_dat['madeg'].unique() # NaNs present

array(['less than high school', 'high school', 'graduate', "bachelor's",
       'associate/junior college'], dtype=object)

In [64]:
gss_dat['padeg'].isnull().sum()
gss_dat['padeg'].unique() # NaNs present

gss_dat = gss_dat.loc[~gss_dat['padeg'].isnull(), ] # keep rows where value for padeg is not NaN
gss_dat['padeg'].unique() # NaNs present

array(['less than high school', "bachelor's", 'high school', 'graduate',
       'associate/junior college'], dtype=object)

In [71]:
gss_dat['sex'].unique() # NaN present
gss_dat = gss_dat.loc[~gss_dat['sex'].isnull(), ] # check codebook, sex == nan does not make sense in context of survey
gss_dat['sex'].unique()

array(['male', 'female'], dtype=object)

In [76]:
gss_dat['race'].value_counts().sum()
gss_dat = gss_dat.loc[~gss_dat['race'].isnull(), ] # keep rows where race is not null

gss_dat['race'].unique()
print("Proportion white:", (gss_dat['race'] == 'white').sum() / (len(gss_dat))) # output of proportions of race in data for reference/Data write up
print("Proportion black:", (gss_dat['race'] == 'black').sum() / (len(gss_dat)))
print("Proportion other than w/b:", (gss_dat['race'] == 'other').sum() / (len(gss_dat)))

Proportion white: 0.8422113861284283
Proportion black: 0.10338867971112894
Proportion other than w/b: 0.054399934160442774


In [93]:
gss_dat = gss_dat.loc[(~gss_dat['wrkstat'].isnull()), ] # remove NaNs
gss_dat['wrkstat'] = gss_dat['wrkstat'].replace('unemployed, laid off, looking for work', 'unemployed') # replace long string of unemployed states with 'unemployed'
(gss_dat['wrkstat'] == 'with a job, but not at work because of temporary illness, vacation, strike').sum() # relatively few entries, annoying value

gss_dat = gss_dat.loc[gss_dat['wrkstat'] != 'with a job, but not at work because of temporary illness, vacation, strike', ] # delete rows w/ this entry ^
print(gss_dat['wrkstat'].value_counts())

working full time    24810
keeping house         6951
retired               6786
working part time     5069
unemployed            1574
in school             1494
other                  858
Name: wrkstat, dtype: int64


In [100]:
gss_dat = gss_dat.loc[~gss_dat['wrkstat'].isnull(), ]
gss_dat['wrkgovt'].unique()

array([nan, 'private', 'government'], dtype=object)

In [ ]:
# Function to max-min normalize data so each variables ranges from 0-1
def maxmin(x):
  res = (x-min(x))/(max(x)-min(x))
  return res

In [ ]:


# Scatter plot grouped by sex, by age and education
plt.scatter()

In [ ]:
# Create scatterplot and run clustering on graph of __ and __ grouped by pres16